In [167]:
from sklearn.preprocessing import StandardScaler
import torch
import torch.nn as nn
from torch.utils.data import DataLoader

import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import sys
import os
from sklearn.preprocessing import MinMaxScaler

import warnings
warnings.filterwarnings('ignore')

np.random.seed(42)

In [168]:
def convert_df_to_numpy(df, scaler):
  y = df['label'].to_numpy().astype(np.int32)
  X = scaler.transform(df.drop(columns=['label']).to_numpy())
  return X, y

center_surround_train_df = pd.read_csv("./center_surround_train.csv")
center_surround_test_df = pd.read_csv("./center_surround_test.csv")
center_surround_valid_df = pd.read_csv("./center_surround_valid.csv")

center_surround_scaler = StandardScaler()
center_surround_scaler.fit(center_surround_train_df.drop(columns=['label']).to_numpy())

center_surround_train_data = convert_df_to_numpy(center_surround_train_df, center_surround_scaler)
center_surround_test_data = convert_df_to_numpy(center_surround_test_df, center_surround_scaler)
center_surround_valid_data = convert_df_to_numpy(center_surround_valid_df, center_surround_scaler)

print(center_surround_train_data[0][:5])
print(center_surround_train_data[1][:5])

[[-1.36846388  1.54362905]
 [ 0.13476095 -0.35808876]
 [-1.17397245  2.0546884 ]
 [ 0.12212715 -0.3837286 ]
 [-0.08095132 -1.53810042]]
[1 0 1 0 1]


In [169]:
INPUT_DIM = 2
OUTPUT_DIM = 1
HIDDEN_NODES = 10 # 2
LEARNING_RATE = 0.1

In [179]:
class FeedForwardNeuralNetwork:
  def __init__(self, 
               train_data, 
               test_data, 
               valid_data, 
               input_dim, 
               output_dim, 
               hidden_nodes, 
               learning_rate, 
               hidden_activation="sigmoid", 
               output_activation='sigmoid'
              ):
    self.train_data = train_data 
    self.test_data = test_data
    self.valid_data = valid_data

    self.input_dim = input_dim
    self.output_dim = output_dim
    self.hidden_nodes = hidden_nodes
    self.learning_rate = learning_rate

    self.parameters = self.initialize_weights()
    self.train()  

  def sigmoid(self, x):
    return 1/(1 + np.exp(-x))
  
  def derivative_sigmoid(self, x):
    return self.sigmoid(x) * (1 - self.sigmoid(x))
  
  def calculate_loss(self):
    predictions = self.parameters['A2'].T
    actuals = self.train_data[1]

    loss = 0
    for i in range(len(predictions)):
      y = actuals[i]
      p = predictions[i][0]
      loss -= y * np.log(p) + (1 - y) * np.log(1 - p)
    loss /= len(actuals)
    return loss

  def initialize_weights(self):
    return {
      'W1': np.random.randn(self.hidden_nodes, self.input_dim),
      'W2': np.random.randn(self.output_dim, self.hidden_nodes)
    }
  
  def train(self):
    EPOCHS = 1000
    for epoch in range(EPOCHS):
      self.forward()

      print(f"Epoch {epoch}/{EPOCHS}   Train Loss: {self.calculate_loss()}")

      gradients = self.backprop()

      self.update_weights(gradients)

  
  def forward(self):
    # Input to Hidden
    self.parameters['A0'] = self.train_data[0]
    self.parameters['Z1'] = np.dot(self.parameters['W1'], self.parameters['A0'].T)
    self.parameters['A1'] = self.sigmoid(self.parameters['Z1'])

    # Hidden to Output
    self.parameters['Z2'] = np.dot(self.parameters['W2'], self.parameters['A1'])
    self.parameters['A2'] = self.sigmoid(self.parameters['Z2']) 
    return self.parameters['A2']
  
  def backprop(self):
    gradients = {}
    predictions = self.parameters['A2']
    actuals = self.train_data[1]

    # Update second weights
    error = predictions - actuals
    gradients['W2'] = np.dot(error, self.parameters['A1'].T)

    # Update first weights
    error = np.multiply( 
      np.dot(self.parameters['W2'].T, error), 
      self.derivative_sigmoid(self.parameters['Z1'])
    )
    gradients['W1'] = np.dot(error, self.parameters['A0'])
    return gradients
  
  def update_weights(self, gradients):
    self.parameters['W1'] -= self.learning_rate * gradients['W1']
    self.parameters['W2'] -= self.learning_rate * gradients['W2']
    

FeedForwardNeuralNetwork(
  train_data=center_surround_train_data,
  test_data=center_surround_test_data,
  valid_data=center_surround_valid_data,
  input_dim=INPUT_DIM,
  output_dim=OUTPUT_DIM,
  hidden_nodes=HIDDEN_NODES,
  learning_rate=LEARNING_RATE
)

Epoch 0/1000   Train Loss: 0.800342468428248
Epoch 1/1000   Train Loss: 4.925635097694572
Epoch 2/1000   Train Loss: 7.647855540943291
Epoch 3/1000   Train Loss: 4.599195921346893
Epoch 4/1000   Train Loss: 7.895096447697492
Epoch 5/1000   Train Loss: 4.056106228047297
Epoch 6/1000   Train Loss: 8.312664460390028
Epoch 7/1000   Train Loss: 3.3164367030282507
Epoch 8/1000   Train Loss: 8.854552458856158
Epoch 9/1000   Train Loss: 2.428144377756813
Epoch 10/1000   Train Loss: nan
Epoch 11/1000   Train Loss: 1.872484257230105
Epoch 12/1000   Train Loss: 5.97253100514434
Epoch 13/1000   Train Loss: 4.787854602716872
Epoch 14/1000   Train Loss: 7.445770057446708
Epoch 15/1000   Train Loss: 3.2099908783015185
Epoch 16/1000   Train Loss: nan
Epoch 17/1000   Train Loss: 1.9404966954944078
Epoch 18/1000   Train Loss: 6.044201413597685
Epoch 19/1000   Train Loss: 4.353310407925704
Epoch 20/1000   Train Loss: 7.654452062171248
Epoch 21/1000   Train Loss: 2.7997965811722803
Epoch 22/1000   Train L